# Install packages

In [ ]:
%%capture

!git clone https://github.com/Deci-AI/super-gradients.git /content/super_gradients_folder
! sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/pretrained_models.py
! sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/utils/checkpoint_utils.py
!pip install -e /content/super_gradients_folder
!pip install onnxruntime

## Select model

In [3]:
model_name = "Models.YOLO_NAS_S" #@param ["Models.YOLO_NAS_S", "Models.YOLO_NAS_M", "Models.YOLO_NAS_L"]
input_width = 320 #@param {type:"slider", min:32, max:4096, step:32}
input_height = 320 #@param {type:"slider", min:32, max:4096, step:32}
dtype = 'fp16' #@param ["uint8", "fp16", "fp32"]

MODEL_FILENAME = f"{model_name}.onnx"
MODEL_UINT8_FILENAME = f"{model_name}_{dtype}_{input_width}x{input_height}.onnx"

In [ ]:
from super_gradients.common.object_names import Models
from super_gradients.training import models

model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, input_height, input_width])

The console stream is logged into /root/sg_logs/console.log


[2025-03-13 17:34:36] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2025-03-13 17:34:43] INFO - utils.py - NumExpr defaulting to 2 threads.
DEBUG:2025-03-13 17:34:49,789:jax._src.path:31: etils.epath found. Using etils.epath for file I/O.


In [63]:
from super_gradients.conversion import DetectionOutputFormatMode

model.export(
  MODEL_FILENAME,
  output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT,
  max_predictions_per_image=20,
  confidence_threshold=0.4,
  input_image_shape=(input_height, input_width)
)

ValueError: It looks like your model does not have dataset preprocessing params properly set.
This may happen if you instantiated model from scratch and not trained it yet. 
Here are what you can do to fix this:
1. Manually fill up dataset processing params via model.set_dataset_processing_params(...).
2. Train your model first and then export it. Trainer will set_dataset_processing_params(...) for you.
3. Instantiate a model using pretrained weights: models.get(..., pretrained_weights="coco") 
4. Disable preprocessing by passing model.export(..., preprocessing=False). 


In [ ]:
import onnxruntime as ort
import numpy as np

dummy_input = np.random.randint(0, 255, (1, 3, input_width, input_height)).astype(np.uint8)

ort_session = ort.InferenceSession(MODEL_FILENAME, providers=["ROCMExecutionProvider"])
ort_session.run(None, {"images": dummy_input})

In [ ]:
from google.colab import files

files.download('yolo_nas_s.onnx')